In [4]:
import kagglehub

!mkdir data

path = kagglehub.dataset_download("dattuan/deutschl-kernscores")

print("Path to dataset files:", path)

mkdir: cannot create directory ‘data’: File exists
Path to dataset files: /root/.cache/kagglehub/datasets/dattuan/deutschl-kernscores/versions/1


In [5]:
!cp -r "/root/.cache/kagglehub/datasets/dattuan/deutschl-kernscores/versions/1" "/content/data/"

In [6]:
import os
import music21 as m21

In [7]:
!mkdir dataset

In [8]:
ACCEPTALE_DURATION = [0.25,0.5,0.75,1.0,1.5,2,3,4]
DATASET_PATH = "/content/data/1"

SAVE_DIR = "/content/dataset"
SINGLE_FILE = "/content/singlefile.txt"


def load_songs_in_kern(dataset_path):
  songs = []
  for path , subdir,files in os.walk(dataset_path):
    for file in files:
      if file.endswith("krn"):
        song = m21.converter.parse(os.path.join(path,file))
        #m21 is used for converting file from one format to another (this is one of the use of that library)
        songs.append(song)
  return songs




def has_acceptable_durations(song , ACCEPTALE_DURATION):
  for note in song.flat.notesAndRests:
    if note.duration.quarterLength not in ACCEPTALE_DURATION:
      return False
  return True


def transpose(song):
  # get kety from song

  parts = song.getElementsByClass(m21.stream.Part)
  measure_part0 = parts[0].getElementsByClass(m21.stream.Measure)
  key = measure_part0[0][4]

  # estimate key from music21
  if not isinstance(key,m21.key.Key):
    key = song.analyze("key")

  # get interval for the transposition
  if key.mode == "major":
    interval = m21.interval.Interval(key.tonic,m21.pitch.Pitch("C"))
  elif key.mode == "minor":
    interval = m21.interval.Interval(key.tonic,m21.pitch.Pitch("A"))

  # transpose the song by calculated interval
  print(key)
  transposed_song = song.transpose(interval)
  return transposed_song


def encode_songs(song,time_step = 0.2):
  encoded_song = []
  # returns song encoded in string
  for event in song.flat.notesAndRests:

    if isinstance(event , m21.note.Note):
      symbol = event.pitch.midi
    elif isinstance(event,m21.note.Rest):
      symbol = "r"
    steps = int(event.duration.quarterLength/time_step)

    for step in range(steps):
      if step == 0:
        encoded_song.append(symbol)
      else:
        encoded_song.append("_")

  encoded_song = " ".join(map(str,encoded_song))
  return encoded_song


def preprocess(dataset_path):
  #load the folk songs
  print("Loadingggg...........")
  songs = load_songs_in_kern(dataset_path )
  print(f"Loaded {len(songs)} songs")

  #filter the songs that have non acceptable durations
  for i,song in enumerate(songs):
    if not has_acceptable_durations(song,ACCEPTALE_DURATION):
      continue

    #transpose c major / a minor
    t_song = transpose(song)

    #encode songs with music time series representation
    encoded_song = encode_songs(t_song)

    #save songs to single text file
    save_path = os.path.join(SAVE_DIR,str(i))
    with open(save_path,"w") as f:
      f.write(encoded_song)

def create_single_file(dataset_path , single_file_path,seq_length = 64):
  new_song_delimiter = "/ " * seq_length
  songs = ""
    # load encoded songs
  for  path,_,files in os.walk(dataset_path):
    for file in files:
      file_path = os.path.join(path,file)
      song = open(file_path,"r").read()

      songs = songs + song + " " + new_song_delimiter

  songs = songs[:-1]


  # save string that contains all dataset
  with open(single_file_path,"w") as f:
    f.write(songs)

  return songs


In [9]:
preprocess(DATASET_PATH)

Loadingggg...........


humdrum.spineParser: WARNING: Error in parsing event ('*MX') at position 8 for spine None: Incorrect meter: *MX found
humdrum.spineParser: WARNING: Error in parsing event ('*MX') at position 8 for spine None: Incorrect meter: *MX found
humdrum.spineParser: WARNING: Error in parsing event ('*MX') at position 8 for spine None: Incorrect meter: *MX found
humdrum.spineParser: WARNING: Error in parsing event ('*MX') at position 8 for spine None: Incorrect meter: *MX found
humdrum.spineParser: WARNING: Error in parsing event ('*MX') at position 8 for spine None: Incorrect meter: *MX found
humdrum.spineParser: WARNING: Error in parsing event ('*MX') at position 8 for spine None: Incorrect meter: *MX found
humdrum.spineParser: WARNING: Error in parsing event ('*MX') at position 8 for spine None: Incorrect meter: *MX found
humdrum.spineParser: WARNING: Error in parsing event ('*M5/1%%%2+3%') at position 52 for spine None: Cannot parse this file -- this error often comes up if the musicxml pickl

Loaded 5365 songs
G major
D major
F major
G major
G major
F major
D major
C major
A- major
G major
B- major
D major
G major
C major


/usr/local/lib/python3.11/dist-packages/music21/stream/base.py:3689: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)


F major
G major
G major
G major
D major
G major
E- major
G major
F major
G major
C major
G major
C major
G major
F major
D major
G major
D major
C major
C major
G major
G major
C major
F major
F major
G major
G major
G major
D major
G major
G major
D major
C major
G major
C major
G major
D major
A major
g minor
G major
B- major
G major
E- major
F major
g minor
G major
e minor
G major
G major
B- major
G major
G major
A major
F major
G major
F major
a minor
F major
C major
C major
C major
G major
A major
F major
D major
G major
G major
G major
G major
G major
D major
F major
G major
F major
G major
G major
G major
G major
G major
D major
F major
G major
C major
G major
G major
A major
G major
G major
C major
F major
E major
F major
C major
B- major
D major
F major
G major
C major
A major
C major
C major
F major
G major
C major
B- major
G major
D major
G major
E major
F major
D major
G major
C major
F major
d minor
B- major
G major
E- major
G major
C major
G major
E major
G major
F major


In [10]:
# put all the songs in single file

songs = create_single_file(SAVE_DIR,SINGLE_FILE)

In [11]:
import json
def mapping(songs,mapping_path):
  #identify the vocab and save them to a json
  mappings = {}
  songs = songs.split()
  vocabulary = list(set(songs))
  for i , symbol in enumerate(vocabulary):
    mappings[symbol] = i

  with open(mapping_path,"w") as f:
    json.dump(mappings,f,indent=4)



In [12]:
MAPPING_PATH = "mapping.json"
mapping(songs,MAPPING_PATH)

In [1]:
# convert the songs to the integers
import numpy as np
import json
import tensorflow.keras as keras
def convert_songs_to_int(songs):
  # load the mappings (json)
  # cast song string to list
  # map songs to int

  int_songs = []

  with open("mapping.json","r") as f :
    mappings = json.load(f)

  songs = songs.split()

  for symbol in songs:
    int_songs.append(mappings[symbol])

  return int_songs

def load(file_path):
    with open(file_path, "r") as fp:
        song = fp.read()
    return song

def generate_training_sequences(sequence_length):
    """Create input and output data samples for training. Each sample is a sequence.

    :param sequence_length (int): Length of each sequence. With a quantisation at 16th notes, 64 notes equates to 4 bars

    :return inputs (ndarray): Training inputs
    :return targets (ndarray): Training targets
    """

    # load songs and map them to int
    songs = load("file_dataset.txt")
    int_songs = convert_songs_to_int(songs)

    inputs = []
    targets = []

    # generate the training sequences
    num_sequences = len(int_songs) - sequence_length
    for i in range(num_sequences):
        inputs.append(int_songs[i:i+sequence_length])
        targets.append(int_songs[i+sequence_length])

    # one-hot encode the sequences
    # the error was here, as the vocab size was too small we change to max
    # to take into account for every possible note
    #vocabulary_size = len(set(int_songs))
    vocabulary_size = max(int_songs) # change is here

    # inputs size: (# of sequences, sequence length, vocabulary size)
    # we also add +1 here to prevent index error
    inputs = keras.utils.to_categorical(inputs, num_classes=vocabulary_size+1) # change is here
    targets = np.array(targets)

    return inputs, targets

In [2]:
Sequencelength = 64
input,target = generate_training_sequences(Sequencelength)

In [3]:
input.shape , target.shape

((2512, 64, 45), (2512,))

In [12]:
import tensorflow.keras as keras

def build_model(op_units,num_units,loss,learning_rate):
  inputs = keras.layers.Input((None,op_units))
  x = keras.layers.LSTM(num_units[0])(inputs)
  x = keras.layers.Dropout(0.2)(x)

  outputs = keras.layers.Dense(op_units,activation="softmax")(x)
  model = keras.Model(inputs,outputs)
  model.compile(loss=loss,optimizer="adam",metrics=["accuracy"])
  model.summary()
  return model


def train():

  OUTPUT_UNITS = 45 # total map size (45 unique char)
  NUM_UNITS = [256]
  LOSS = "sparse_categorical_crossentropy"
  LR = 0.001
  EPOCHS = 45
  BATCH_SIZE = 256

  #build model
  model = build_model(OUTPUT_UNITS,NUM_UNITS,LOSS,LR)

  # train the model
  model.fit(input,target,batch_size=BATCH_SIZE,epochs=EPOCHS)

  # save the model
  model.save("model.keras")

In [13]:
train()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, None, 45)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 256)                 │         309,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 45)                  │          11,565 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 320,813 (1.22 MB)

 Trainable params: 320,813 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/45
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 513ms/step - accuracy: 0.4768 - loss: 3.3303
Epoch 2/45
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 493ms/step - accuracy: 0.6508 - loss: 1.5647
Epoch 3/45
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 462ms/step - accuracy: 0.7255 - loss: 1.2690
Epoch 4/45
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 462ms/step - accuracy: 0.7440 - loss: 1.1735
Epoch 5/45
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 523ms/step - accuracy: 0.7325 - loss: 1.1909
Epoch 6/45
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 521ms/step - accuracy: 0.7386 - loss: 1.1472
Epoch 7/45
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 462ms/step - accuracy: 0.7461 - loss: 1.1223
Epoch 8/45
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 488ms/step - accuracy: 0.7474 - loss: 1.1336
Epoch 9/45
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 472ms/step - accuracy: 0.7525 - loss: 1.1110
Epoch 10/45
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 458ms/step - accuracy: 0.7551 - loss: 1.0962
Epoch 11/45
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 515ms/step - accuracy: 0.7511 - loss: 1.0771
Epoch 12/45
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 462ms/st

In [24]:
import tensorflow.keras as keras

class Melodygenerator():
  def __init__(self,model_path):
    self.model_path = model_path
    self.model = keras.models.load_model("model.keras")

    with open("mapping.json","r") as f:
      self.mappings = json.load(f)

    self.start_symbols = ["/"] * Sequencelength


  def sample_with_temperature(self,probabilities,temperature):
    # if temp -> infinity all the values tends to have uniform pdf
    # if temp -> 0 then the particular value is totally dominant over all approx = 1
    predictions = np.log(probabilities) / temperature
    probabilities = np.exp(predictions)/np.sum(np.exp(predictions))
    choices = range(len(probabilities))
    index = np.random.choice(choices,p=probabilities) # to each item of the choices we have their corresponding prob output that

    return index



  def generate_melody(self,seed,num_steps,max_seqlen,temperature):

    # create seed with start symbols
    seed = seed.split()
    melody = seed
    seed = self.start_symbols + seed

    # map seed to int
    seed= [self.mappings[symbols] for symbols in seed]

    for _ in range(num_steps):
      # limit the seed to max_seq_len
      seed_list  = seed[-max_seqlen:] # last relevent steps are considered
      #one hot encoded
      oh_seed = keras.utils.to_categorical(seed_list,num_classes=len(self.mappings))
      #to make a prediction we have to add extra dimension
      oh_seed = oh_seed[np.newaxis,...]


      probabilities = self.model.predict(oh_seed)[0]
      output_int = self.sample_with_temperature(probabilities,temperature)

      seed.append(output_int)

      output_symbol = [k for k,v in self.mappings.items() if v == output_int][0]

      # we are using temperature sampling over just showing the highest prob value

      # CHECK if we are at the end of the song if so then exit
      if output_symbol=="/":
        break

      melody.append(output_symbol)
    return melody





In [25]:
mlg = Melodygenerator("model.keras")
seed = "55 _ _ _ 64 _ _ _ 64 _ 64"
melody = mlg.generate_melody(seed,num_steps = 500, max_seqlen=  64, temperature = 0.7)
print(melody)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━

In [28]:
# Visualize the melody
import music21 as m21
def save_melody(melody,format = "midi",file_name = "mel.midi",step_duration = 0.25):

  # create a music 21 stream
  stream = m21.stream.Stream()

  # parse all the symbols in melody and create note/rest objects
  start_symbol = None
  step_counter = 1

  for i,symbol in enumerate(melody):

    # we have note/rest
    if symbol != "_" or i+1 ==len(melody):
      if start_symbol is not None:
        quarter_length_duration = step_duration * step_counter
        # if rest
        if start_symbol == "r":
          m21_event = m21.note.Rest(quarterLength=quarter_length_duration)
        # if note
        else:
          m21_event = m21.note.Note(int(start_symbol),quarterLength=quarter_length_duration)
        stream.append(m21_event)
        # reset
        step_counter = 1
      start_symbol = symbol

    # we have "_"  it is prolongation sign
    else:
      step_counter+=1

  stream.write(format,file_name)





In [29]:
save_melody(melody)